# Ритейл — Анализ программы лояльности

## Задачи проекта:

Проанализировать программу лояльности магазина.

Провести исследовательский анализ данных;\
Провести анализ программы лояльности;\
Сформулировать и проверить статистические гипотезы.\
Проверить гипотезу: средний чек участников программы лояльности выше, чем у остальных покупателей.\
Сформулировать собственную ретроспективную статистическую гипотезу. Дополнить ее нулевой и альтернативной гипотезами.\
Проверить гипотезы с помощью статистического теста.

## Описание данных:
Датасет содержит данные о покупках в магазине строительных материалов «Строили, строили и наконец построили». \
Все покупатели могут получить доступ в магазин с использованием персональных карт. \
За 200 рублей в месяц они могут стать участниками программы лояльности. \
В программу включены скидки, специальные предложения, подарки.

### Стадии проекта:
предобработка данных;\
исследовательский анализ данных;\
проверка гипотез;\
работа с метриками и бизнес-показателями;\
подготовка презентации.

### Поведенческие алгоритмы
прояснение запроса;\
декомпозиция задачи;\
работа с документацией,\
презентация результатов,\
и другие.

### Требования заказчика уточняют, а саму задачу декомпозируют, или разбивают на этапы.

In [1]:
#загрузим нужные библиотеки
import pandas as pd
import numpy as np
from numpy import mean
import math as mth
import datetime as dt
import seaborn as sns
import scipy.stats as st
import plotly.express as px
import matplotlib.pyplot as plt
from plotly import graph_objects as go

In [2]:
#загрузим данные в переменную 'data'

product_codes = \
pd.read_csv("C:\\Users\\kirio\\OneDrive\\Рабочий стол\\ЯПрактикум\\14_Выпускной Проект\\product_codes.csv", sep=',')

retail_dataset = \
pd.read_csv('C:\\Users\\kirio\\OneDrive\\Рабочий стол\\ЯПрактикум\\14_Выпускной Проект\\retail_dataset.csv', sep=',')
  

In [3]:
product_codes

,productID,price_per_one
0,85123A,2.55
1,71053,3.39
2,84406B,2.75
3,84029G,3.39
4,84029E,3.39
...,...,...
9964,84306,8.29
9965,90001A,4.15
9966,90118,2.46
9967,DOT,172.62


In [5]:
product_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9969 entries, 0 to 9968
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   productID      9969 non-null   object 
 1   price_per_one  9969 non-null   float64
dtypes: float64(1), object(1)
memory usage: 155.9+ KB


In [8]:
product_codes.describe()

,price_per_one
count,9969.000000
mean,19.503697
std,330.880754
min,0.000000
25%,1.250000
50%,2.550000
75%,5.510000
max,16888.020000


product_codes.csv:

- `productID` — id товара;
- `price_per_one` — стоимость одной единицы товара;

In [4]:
retail_dataset

,purchaseid,item_ID,Quantity,purchasedate,CustomerID,ShopID,loyalty_program
0,538280,21873,11,2016-12-10 12:50:00,18427.0,Shop 0,0.0
1,538862,22195,0,2016-12-14 14:11:00,22389.0,Shop 0,1.0
2,538855,21239,7,2016-12-14 13:50:00,22182.0,Shop 0,1.0
3,543543,22271,0,2017-02-09 15:33:00,23522.0,Shop 0,1.0
4,543812,79321,0,2017-02-13 14:40:00,23151.0,Shop 0,1.0
...,...,...,...,...,...,...,...
105330,538566,21826,1,2016-12-13 11:21:00,NaN,Shop 0,0.0
105331,540247,21742,0,2017-01-05 15:56:00,21143.0,Shop 0,0.0
105332,538068,85048,1,2016-12-09 14:05:00,23657.0,Shop 0,1.0
105333,538207,22818,11,2016-12-10 11:33:00,18427.0,Shop 0,0.0


In [6]:
retail_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105335 entries, 0 to 105334
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   purchaseid       105335 non-null  object 
 1   item_ID          105335 non-null  object 
 2   Quantity         105335 non-null  int64  
 3   purchasedate     105335 non-null  object 
 4   CustomerID       69125 non-null   float64
 5   ShopID           105335 non-null  object 
 6   loyalty_program  105335 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 5.6+ MB


retail_dataset.csv:

- `purchaseId` — id чека;
- `item_ID` — id товара;
- `purchasedate` — дата покупки;
- `Quantity` — количество товара;
- `CustomerID` — id покупателя;
- `ShopID` — id магазина;
- `loyalty_program` — участвует ли покупатель в программе лояльности;

In [7]:
retail_dataset.describe()

,Quantity,CustomerID,loyalty_program
count,105335.000000,69125.000000,105335.000000
mean,7.821218,21019.302047,0.226345
std,327.946695,1765.444679,0.418467
min,-74216.000000,18025.000000,0.000000
25%,0.000000,19544.000000,0.000000
50%,2.000000,20990.000000,0.000000
75%,7.000000,22659.000000,0.000000
max,74214.000000,23962.000000,1.000000


# Предобработка данных

Исследовать пропущенные значения;\
Исследовать соответствие типов;\
Исследовать дубликаты;\
Проверить корректность наименований колонок;\
Переименовать колонки;\
Удалить дубликаты;\
Привести типы;\
Заменить пропущенные значения.

# Исследовательский анализ данных

### Вопросы к тимлиду:
Кто заказчик?\
Какие решения будут принимать на основе проведённого анализа?\
Как собирали данные?\
В данных какая-то аномалия, что бы это могло значить?\
Это дубликаты или так и должно быть?\
Как данные в столбце соотносятся с контекстом задачи?\
Что вообще требуется сделать?\
Как правильно рассчитать показатель?

Если вы обнаружили аномалию или дубликаты, отправьте ревьюеру конкретный пример.